<a href="https://colab.research.google.com/github/sneharaoganta/Recipe_Generator/blob/main/recipeGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Packages for training the model and working with the dataset.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import json

# Utility/helper packages.
import platform
import time
import pathlib
import os

In [2]:
print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)


Python version: 3.7.10
Tensorflow version: 2.4.1
Keras version: 2.4.0


In [3]:
def load_dataset(silent=False):
    # List of dataset files we want to merge.
    dataset_file_names = [
        'recipes_with_nutritional_info.json'
    ]
    

    
    dataset = []
    for dataset_file_name in dataset_file_names:
        dataset_file_path = f'/content/drive/MyDrive/{dataset_file_name}'
        with open(dataset_file_path) as dataset_file:
            json_data_list = json.load(dataset_file)
            dict_keys = [key for key in json_data_list[0]]
            dict_keys.sort()
            dataset += json_data_list
            # This code block outputs the summary for each dataset.
            if silent == False:
                print(dataset_file_path)
                print('===========================================')
                print('Number of examples: ', len(json_data_list), '\n')
                print('Example object keys:\n', dict_keys, '\n')
                print('Example object:\n', json_data_list[0], '\n')
                print('Required keys:\n')
                print('  title: ', json_data_list[0]['title'], '\n')
                print('  ingredients: ', json_data_list[0]['ingredients'], '\n')
                print('  instructions: ', json_data_list[0]['instructions'])
                print('\n\n')
    return dataset  
dataset_raw = load_dataset()

/content/drive/MyDrive/recipes_with_nutritional_info.json
Number of examples:  51235 

Example object keys:
 ['fsa_lights_per100g', 'id', 'ingredients', 'instructions', 'nutr_per_ingredient', 'nutr_values_per100g', 'partition', 'quantity', 'title', 'unit', 'url', 'weight_per_ingr'] 

Example object:
 {'fsa_lights_per100g': {'fat': 'green', 'salt': 'green', 'saturates': 'green', 'sugars': 'orange'}, 'id': '000095fc1d', 'ingredients': [{'text': 'yogurt, greek, plain, nonfat'}, {'text': 'strawberries, raw'}, {'text': 'cereals ready-to-eat, granola, homemade'}], 'instructions': [{'text': 'Layer all ingredients in a serving dish.'}], 'nutr_per_ingredient': [{'fat': 0.8845044000000001, 'nrg': 133.80964, 'pro': 23.110512399999998, 'sat': 0.26535132, 'sod': 81.64656, 'sug': 7.348190400000001}, {'fat': 0.46, 'nrg': 49.0, 'pro': 1.02, 'sat': 0.023, 'sod': 2.0, 'sug': 7.43}, {'fat': 7.415, 'nrg': 149.25, 'pro': 4.17, 'sat': 1.207, 'sod': 8.0, 'sug': 6.04}], 'nutr_values_per100g': {'energy': 81.12

In [4]:
def recipe_validate_required_fields(recipe):
    required_keys = ['title', 'ingredients', 'instructions']
    
    if not recipe:
        return False
    
    for required_key in required_keys:
        if not recipe[required_key]:
            return False
        
        if type(recipe[required_key]) == list and len(recipe[required_key]) == 0:
            return False
    
    return True

In [5]:
dataset_validated = [recipe for recipe in dataset_raw if recipe_validate_required_fields(recipe)]
print('Dataset size BEFORE validation', len(dataset_raw))
print('Dataset size AFTER validation', len(dataset_validated))
print('Number of incomplete recipes', len(dataset_raw) - len(dataset_validated))

Dataset size BEFORE validation 51235
Dataset size AFTER validation 51235
Number of incomplete recipes 0


In [6]:
def recipe_to_string(recipe):
    # This string is presented as a part of recipes so we need to clean it up.
    # noize_string = 'ADVERTISEMENT'
    
    title = recipe['title']
    ingredients = recipe['ingredients']
    instructions = recipe['instructions']
    
    ingredients_string = ''
    for ingredient in ingredients:
        ingredient = ingredient['text']
        if ingredient:
            ingredients_string += f'• {ingredient}\n'
    
    instructions_string = ''
    for instruction in instructions:
        instruction = instruction['text']
        if instruction:
            instructions_string += f'▪︎ {instruction}\n'
    
    return f'{STOP_WORD_TITLE}{title}\n{STOP_WORD_INGREDIENTS}{ingredients_string}{STOP_WORD_INSTRUCTIONS}{instructions_string}'

In [7]:
STOP_WORD_TITLE = 'titlename' 
STOP_WORD_INGREDIENTS = 'ingname'
STOP_WORD_INSTRUCTIONS = 'insts'

In [8]:
dataset_stringified = [recipe_to_string(recipe) for recipe in dataset_validated]
print('Stringified dataset size: ', len(dataset_stringified))

Stringified dataset size:  51235


In [9]:
for recipe_index, recipe_string in enumerate(dataset_stringified[:3]):
    print('Recipe #{}\n---------'.format(recipe_index + 1))
    print(recipe_string)
    print('\n')

Recipe #1
---------
titlenameYogurt Parfaits
ingname• yogurt, greek, plain, nonfat
• strawberries, raw
• cereals ready-to-eat, granola, homemade
insts▪︎ Layer all ingredients in a serving dish.



Recipe #2
---------
titlenameSalt Free, Low Cholesterol Sugar Cookies Recipe
ingname• sugars, granulated
• oil, corn, peanut, and olive
• egg substitute, powder
• orange juice, raw
• orange juice, raw
• leavening agents, baking powder, double-acting, sodium aluminum sulfate
• wheat flour, white, all-purpose, unenriched
insts▪︎ Cream sugar and butter together till smooth.
▪︎ Add in egg beaters, orange rind, orange juice, and mix well.
▪︎ Mix together low sodium baking powder and flour.
▪︎ Add in to creamed mix and mix well.
▪︎ Roll dough into 1 inch balls and place on ungreased cookie sheet.
▪︎ Rub small amount of salt free butter on bottom of glass.
▪︎ Dip glass in granulated sugar.
▪︎ Flatten cookie dough ball slightly using flat end of glass.
▪︎ Bake at 300 degrees for 10-12 min.



Recipe 

In [10]:
print(dataset_stringified[50000])

titlenamePumpkin Oatmeal
ingname• oats
• milk, fluid, 1% fat, without added vitamin a and vitamin d
• pumpkin, raw
• pumpkin, raw
• sugars, granulated
insts▪︎ Mix together oats and milk in a microwave-safe bowl.
▪︎ Cook on high for 1 to 2 minutes, stirring once.
▪︎ Add more milk or oats to achieve the desired consistency, and cook for another 30 seconds.
▪︎ Stir in pumpkin puree, pumpkin pie spice, and cinnamon sugar.
▪︎ Heat through, and serve.



In [11]:
MAX_RECIPE_LENGTH = 2000

In [12]:
def filter_recipes_by_length(recipe_test):
    return len(recipe_test) <= MAX_RECIPE_LENGTH 
dataset_filtered = [recipe_text for recipe_text in dataset_stringified if filter_recipes_by_length(recipe_text)]
print('Dataset size BEFORE filtering: ', len(dataset_stringified))
print('Dataset size AFTER filtering: ', len(dataset_filtered))
print('Number of eliminated recipes: ', len(dataset_stringified) - len(dataset_filtered))

Dataset size BEFORE filtering:  51235
Dataset size AFTER filtering:  50217
Number of eliminated recipes:  1018


In [13]:
TOTAL_RECIPES_NUM = len(dataset_filtered)
print('MAX_RECIPE_LENGTH: ', MAX_RECIPE_LENGTH)
print('TOTAL_RECIPES_NUM: ', TOTAL_RECIPES_NUM)

MAX_RECIPE_LENGTH:  2000
TOTAL_RECIPES_NUM:  50217


In [14]:
STOP_SIGN = '␣'
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=True,
    filters='',
    lower=False,
    split=''
)
# Stop word is not a part of recipes, but tokenizer must know about it as well.
tokenizer.fit_on_texts([STOP_SIGN])
tokenizer.fit_on_texts(dataset_filtered)
tokenizer.get_config()

{'char_level': True,
 'document_count': 50218,
 'filters': '',
 'index_docs': '{"1": 50217, "100": 1, "35": 32291, "27": 47349, "18": 50217, "4": 50217, "7": 50216, "2": 50217, "11": 50200, "22": 50217, "24": 49637, "72": 4607, "8": 50216, "23": 50217, "29": 47017, "58": 12293, "9": 50217, "12": 50083, "6": 50217, "28": 47728, "17": 50217, "15": 50217, "20": 49947, "26": 50217, "3": 50217, "16": 50215, "10": 50217, "21": 49575, "19": 49705, "34": 31407, "14": 50096, "25": 47631, "13": 49966, "5": 50217, "52": 15561, "30": 32764, "46": 23796, "38": 30123, "45": 19392, "41": 24779, "37": 27280, "33": 37071, "32": 38645, "50": 19324, "31": 34107, "36": 27191, "39": 24467, "43": 22911, "54": 14472, "74": 3731, "63": 11007, "48": 21025, "51": 17104, "47": 21105, "44": 20892, "71": 5321, "40": 23351, "42": 21284, "64": 11065, "49": 15491, "68": 6327, "53": 12818, "65": 6983, "67": 9042, "55": 9950, "60": 10507, "66": 8496, "70": 5563, "56": 12345, "57": 9885, "69": 7209, "62": 9194, "76": 20

In [15]:
VOCABULARY_SIZE = len(tokenizer.word_counts) + 1
array_vocabulary = tokenizer.sequences_to_texts([[word_index] for word_index in range(VOCABULARY_SIZE)])
print('vocabulary size: ', VOCABULARY_SIZE)
dataset_vectorized = tokenizer.texts_to_sequences(dataset_filtered)
print('Vectorized dataset size', len(dataset_vectorized))

vocabulary size:  101
Vectorized dataset size 50217


In [16]:
def recipe_sequence_to_string(recipe_sequence):
    recipe_stringified = tokenizer.sequences_to_texts([recipe_sequence])[0]
    print(recipe_stringified)
recipe_sequence_to_string(dataset_vectorized[0])

t i t l e n a m e Y o g u r t   P a r f a i t s 
 i n g n a m e •   y o g u r t ,   g r e e k ,   p l a i n ,   n o n f a t 
 •   s t r a w b e r r i e s ,   r a w 
 •   c e r e a l s   r e a d y - t o - e a t ,   g r a n o l a ,   h o m e m a d e 
 i n s t s ▪ ︎   L a y e r   a l l   i n g r e d i e n t s   i n   a   s e r v i n g   d i s h . 



In [17]:
for recipe_index, recipe in enumerate(dataset_vectorized[:10]):
    print('Recipe #{} length: {}'.format(recipe_index + 1, len(recipe)))

Recipe #1 length: 174
Recipe #2 length: 773
Recipe #3 length: 1118
Recipe #4 length: 541
Recipe #5 length: 660
Recipe #6 length: 411
Recipe #7 length: 420
Recipe #8 length: 1368
Recipe #9 length: 1432
Recipe #10 length: 785


In [18]:
dataset_vectorized_padded_without_stops = tf.keras.preprocessing.sequence.pad_sequences(
    dataset_vectorized,
    padding='post',
    truncating='post',
   
    maxlen=MAX_RECIPE_LENGTH-1,
    value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)
dataset_vectorized_padded = tf.keras.preprocessing.sequence.pad_sequences(
    dataset_vectorized_padded_without_stops,
    padding='post',
    truncating='post',
    maxlen=MAX_RECIPE_LENGTH+1,
    value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)
for recipe_index, recipe in enumerate(dataset_vectorized_padded[:10]):
    print('Recipe #{} length: {}'.format(recipe_index, len(recipe)))

Recipe #0 length: 2001
Recipe #1 length: 2001
Recipe #2 length: 2001
Recipe #3 length: 2001
Recipe #4 length: 2001
Recipe #5 length: 2001
Recipe #6 length: 2001
Recipe #7 length: 2001
Recipe #8 length: 2001
Recipe #9 length: 2001


In [19]:
recipe_sequence_to_string(dataset_vectorized_padded[0])

t i t l e n a m e Y o g u r t   P a r f a i t s 
 i n g n a m e •   y o g u r t ,   g r e e k ,   p l a i n ,   n o n f a t 
 •   s t r a w b e r r i e s ,   r a w 
 •   c e r e a l s   r e a d y - t o - e a t ,   g r a n o l a ,   h o m e m a d e 
 i n s t s ▪ ︎   L a y e r   a l l   i n g r e d i e n t s   i n   a   s e r v i n g   d i s h . 
 ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ ␣ 

In [20]:
dataset = tf.data.Dataset.from_tensor_slices(dataset_vectorized_padded)
print(dataset)

<TensorSliceDataset shapes: (2001,), types: tf.int32>


In [21]:
def split_input_target(recipe):
    input_text = recipe[:-1]
    target_text = recipe[1:]
    
    return input_text, target_text
dataset_targeted = dataset.map(split_input_target)
print(dataset_targeted)

<MapDataset shapes: ((2000,), (2000,)), types: (tf.int32, tf.int32)>


In [22]:
for input_example, target_example in dataset_targeted.take(1):
    print('Input sequence size:', repr(len(input_example.numpy())))
    print('Target sequence size:', repr(len(target_example.numpy())))
    
    
    input_stringified = tokenizer.sequences_to_texts([input_example.numpy()[:50]])[0]
    target_stringified = tokenizer.sequences_to_texts([target_example.numpy()[:50]])[0]
    
    

Input sequence size: 2000
Target sequence size: 2000


In [23]:

BATCH_SIZE = 32

SHUFFLE_BUFFER_SIZE = 1000
dataset_train = dataset_targeted \
.shuffle(SHUFFLE_BUFFER_SIZE) \
.batch(BATCH_SIZE, drop_remainder=True) \
.repeat()
print(dataset_train)

<RepeatDataset shapes: ((32, 2000), (32, 2000)), types: (tf.int32, tf.int32)>


In [24]:
for input_text, target_text in dataset_train.take(1):
    print('1st batch: input_text:', input_text)
    print()
    print('1st batch: target_text:', target_text)

1st batch: input_text: tf.Tensor(
[[  3   6   3 ... 100 100 100]
 [  3   6   3 ... 100 100 100]
 [  3   6   3 ... 100 100 100]
 ...
 [  3   6   3 ... 100 100 100]
 [  3   6   3 ... 100 100 100]
 [  3   6   3 ... 100 100 100]], shape=(32, 2000), dtype=int32)

1st batch: target_text: tf.Tensor(
[[  6   3  10 ... 100 100 100]
 [  6   3  10 ... 100 100 100]
 [  6   3  10 ... 100 100 100]
 ...
 [  6   3  10 ... 100 100 100]
 [  6   3  10 ... 100 100 100]
 [  6   3  10 ... 100 100 100]], shape=(32, 2000), dtype=int32)


In [25]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        batch_input_shape=[batch_size, None]
    ))
    model.add(tf.keras.layers.LSTM(
        units=rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer=tf.keras.initializers.GlorotNormal()
    ))
    model.add(tf.keras.layers.Dense(vocab_size))
    
    return model
model = build_model(
  vocab_size=VOCABULARY_SIZE,
  embedding_dim=256,
  rnn_units=512,
  batch_size=BATCH_SIZE
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           25856     
_________________________________________________________________
lstm (LSTM)                  (32, None, 512)           1574912   
_________________________________________________________________
dense (Dense)                (32, None, 101)           51813     
Total params: 1,652,581
Trainable params: 1,652,581
Non-trainable params: 0
_________________________________________________________________


In [26]:
for input_example_batch, target_example_batch in dataset_train.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 2000, 101) # (batch_size, sequence_length, vocab_size)


In [27]:
tmp_logits = [
  [-0.95, 0, 0.95],
];

tmp_samples = tf.random.categorical(
    logits=tmp_logits,
    num_samples=5
)
print(tmp_samples)

tf.Tensor([[1 1 2 2 2]], shape=(1, 5), dtype=int64)


In [28]:
sampled_indices = tf.random.categorical(
    logits=example_batch_predictions[0],
    num_samples=1
)
sampled_indices = tf.squeeze(
    input=sampled_indices,
    axis=-1
).numpy()
sampled_indices.shape

(2000,)

In [29]:
# An objective function.
# The function is any callable with the signature scalar_loss = fn(y_true, y_pred).
def loss(labels, logits):
    entropy = tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )
    
    return entropy

    model.compile( optimizer=tf.keras.optimizers.RMSprop(),loss=loss)


In [30]:
EPOCHS = 15
INITIAL_EPOCH = 1
STEPS_PER_EPOCH = 250


adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam_optimizer,
    loss=loss
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=5,
    monitor='loss',
    restore_best_weights=True,
    verbose=1
)

In [31]:
chkpnt_dir = '/content/gdrive/MyDrive'
os.makedirs(chkpnt_dir, exist_ok=True)
chkpnt_pref = os.path.join(chkpnt_dir, 'ckpt_{epoch}')
chkpnt_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=chkpnt_pref,
    save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

In [ ]:
history = model.fit(
    x=dataset_train,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    initial_epoch=INITIAL_EPOCH,
    callbacks=[
        early_stopping_callback
    ]
)
# Saving the trained model to file (to be able to re-use it later).
checkpoint_dir="/content/gdrive/MyDrive"
os.listdir(checkpoint_dir)
model_name = 'recipe_maker.h5'
model.save(model_name, save_format='h5')


Epoch 2/15
250/250 [==============================] - 73s 285ms/step - loss: 1.3447
Epoch 3/15
250/250 [==============================] - 73s 293ms/step - loss: 0.6685
Epoch 4/15
250/250 [==============================] - 74s 297ms/step - loss: 0.5002
Epoch 5/15
250/250 [==============================] - 74s 297ms/step - loss: 0.3960
Epoch 6/15
250/250 [==============================] - 74s 295ms/step - loss: 0.3450
Epoch 7/15
250/250 [==============================] - 74s 296ms/step - loss: 0.3044
Epoch 8/15
250/250 [==============================] - 74s 296ms/step - loss: 0.2859
Epoch 9/15
250/250 [==============================] - 74s 296ms/step - loss: 0.2761
Epoch 10/15
250/250 [==============================] - 74s 296ms/step - loss: 0.2617
Epoch 11/15
250/250 [==============================] - 74s 296ms/step - loss: 0.2537
Epoch 12/15
250/250 [==============================] - 74s 296ms/step - loss: 0.2465
Epoch 13/15
250/250 [==============================] - 74s 295ms/step - l

In [32]:
def generate_text(model, start_string, num_generate = 1000, temperature=1.0):
    # Evaluation step (generating text using the learned model)
    
    padded_start_string = STOP_WORD_TITLE + start_string
    # Converting our start string to numbers (vectorizing).
    input_indices = np.array(tokenizer.texts_to_sequences([padded_start_string]))
    # Empty string to store our results.
    text_generated = []
    # Here batch size == 1.
    model.reset_states()
    for char_index in range(num_generate):
        predictions = model(input_indices)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        # Using a categorical distribution to predict the character returned by the model.
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            predictions,
            num_samples=1
        )[-1, 0].numpy()
        # We pass the predicted character as the next input to the model
        # along with the previous hidden state.
        input_indices = tf.expand_dims([predicted_id], 0)
        
        next_character = tokenizer.sequences_to_texts(input_indices.numpy())[0]
        text_generated.append(next_character)
    return (padded_start_string + ''.join(text_generated))

In [33]:
def generate_combinations(model):
    recipe_length = 1000
    try_letters = ['', '\n', 'A', 'B', 'C', 'O', 'L', 'Mushroom', 'Apple', 'Slow', 'Christmass', 'The', 'Banana', 'Homemade']
    try_temperature = [1.0, 0.8, 0.4, 0.2]
    for letter in try_letters:
        for temperature in try_temperature:
            generated_text = generate_text(
                model,
                start_string=letter,
                num_generate = recipe_length,
                temperature=temperature
            )
            print(f'Attempt: "{letter}" + {temperature}')
            print('-----------------------------------')
            print(generated_text)
            print('\n\n')

In [34]:

simplified_batch_size = 1
checkpoint_dir = "/content/drive/MyDrive/recipe_maker.h5"
model_simplified = build_model(
  vocab_size=VOCABULARY_SIZE,
  embedding_dim=256,
  rnn_units=512,
  batch_size=simplified_batch_size)
model_simplified.load_weights(checkpoint_dir)
model_simplified.build(tf.TensorShape([simplified_batch_size, None]))
model_simplified.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            25856     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 512)            1574912   
_________________________________________________________________
dense_1 (Dense)              (1, None, 101)            51813     
Total params: 1,652,581
Trainable params: 1,652,581
Non-trainable params: 0
_________________________________________________________________


In [35]:
generate_combinations(model_simplified)

Attempt: "" + 1.0
-----------------------------------
titlenamez>
}␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣␣